# Fingerprints Quality Classification
## DNN Course JCE - Final Project 
### By Kobi Amsellem & Zohar Kedem

Expirements:
* Try to classify 5 classes that represat the quality of the finger print (0 poor, 4 best)

In [2]:
import numpy as np
import tensorflow as tf
import keras
import pathlib

In [5]:
dataset_dir = pathlib.Path('./Dataset')
count = len(list(dataset_dir.glob('*/*.jpg')))
print(count)


976
